In [25]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_error_measures
from src.models import TwoLayerPerceptron, TwoLayerLeaky, TwoLayerSynaptic, ThreeLayerLeaky
from src.train_model import training_loop

In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [18]:
np.random.seed(373)
torch.manual_seed(373)

In [19]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [20]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # normalization
train_dataset, test_dataset = torch.utils.data.random_split(all_data, (round(0.4 * len(all_data)), round(0.6 * len(all_data))))

In [21]:
#train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)

### Two layer perceptron

In [26]:
for i in range(5):
    h = np.random.randint(500, 3000)
    e = np.random.randint(20, 200)
    b = np.random.choice([16, 32, 64, 128])
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, batch size: {b}')
    train_loader = data.DataLoader(train_dataset, batch_size=int(b), shuffle=True, drop_last=True)
    model = TwoLayerPerceptron(num_inputs=7, num_hidden=h, num_outputs=1)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, validation=False)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 2583, epochs: 139, batch size: 128


100%|██████████| 139/139 [00:11<00:00, 11.67it/s]


Accuracy on validation dataset: 42.76632633403968%
MAE: 6.122987860641143, MSE:79.4420617277969,
RMSE: 8.913027640919605, Index of Agreement: 0.8871940462680998

------------- Random set: 1 -----------------
Hidden: 1861, epochs: 195, batch size: 16


100%|██████████| 195/195 [01:37<00:00,  2.00it/s]


Accuracy on validation dataset: 43.80658832594876%
MAE: 5.926686568948184, MSE:76.21347561130275,
RMSE: 8.730032967366318, Index of Agreement: 0.8885127349633807

------------- Random set: 2 -----------------
Hidden: 763, epochs: 51, batch size: 16


...

Accuracy on validation dataset: 40.570217684453866%
MAE: 6.349217872127268, MSE:84.89538790284851,
RMSE: 9.213869323082921, Index of Agreement: 0.8730362666868916

------------- Random set: 3 -----------------
Hidden: 1717, epochs: 33, batch size: 128


100%|██████████| 33/33 [00:02<00:00, 11.77it/s]


Accuracy on validation dataset: 39.587748025428624%
MAE: 6.61672124847055, MSE:91.93381507286574,
RMSE: 9.588212298070259, Index of Agreement: 0.864596351022823

------------- Random set: 4 -----------------
Hidden: 1758, epochs: 83, batch size: 16


100%|██████████| 83/83 [00:40<00:00,  2.07it/s]

Accuracy on validation dataset: 40.45463301868619%
MAE: 6.226557504020429, MSE:81.41750360820298,
RMSE: 9.023164833261275, Index of Agreement: 0.8778917210597745


### SNN with LIF neurons

In [22]:
for i in range(5):
    h = np.random.randint(1000, 2500)
    e = np.random.randint(20, 50)
    s = np.random.randint(20, 30)
    b = np.random.choice([32, 64])
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, steps: {s}, batch size: {b}')
    train_loader = data.DataLoader(train_dataset, batch_size=int(b), shuffle=True, drop_last=True)
    model = TwoLayerLeaky(num_inputs=7, num_hidden=h, num_outputs=1, beta=0.95, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, validation=False)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 1973, epochs: 23, steps: 23, batch size: 32


100%|██████████| 23/23 [00:50<00:00,  2.19s/it]


Accuracy on validation dataset: 38.45116547871316%
MAE: 6.929131181087137, MSE:98.79313193820037,
RMSE: 9.939473423587406, Index of Agreement: 0.8500468653125463

------------- Random set: 1 -----------------
Hidden: 2398, epochs: 34, steps: 23, batch size: 64


100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


Accuracy on validation dataset: 38.932768252745134%
MAE: 6.866106725891898, MSE:98.50688131193391,
RMSE: 9.925063290071954, Index of Agreement: 0.8507236827152364

------------- Random set: 2 -----------------
Hidden: 1071, epochs: 43, steps: 26, batch size: 32


...

Accuracy on validation dataset: 38.45116547871316%
MAE: 6.904280479508681, MSE:101.25382162002806,
RMSE: 10.062495794783125, Index of Agreement: 0.8432570857401828

------------- Random set: 3 -----------------
Hidden: 1116, epochs: 32, steps: 24, batch size: 64


100%|██████████| 32/32 [00:35<00:00,  1.11s/it]


Accuracy on validation dataset: 37.14120593334618%
MAE: 7.177809658479516, MSE:107.18825281208647,
RMSE: 10.353175977065515, Index of Agreement: 0.8313244801154727

------------- Random set: 4 -----------------
Hidden: 1151, epochs: 33, steps: 25, batch size: 64


100%|██████████| 33/33 [00:38<00:00,  1.16s/it]


Accuracy on validation dataset: 37.603544596416874%
MAE: 6.984923908462921, MSE:102.36238043588122,
RMSE: 10.11742953698622, Index of Agreement: 0.8423753684443211


### SNN with Synaptic neurons

In [23]:
for i in range(2):
    h = np.random.randint(1000, 3000)
    e = np.random.randint(10, 40)
    #s = np.random.randint(80, 110)
    #b = np.random.choice([16, 32])
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}')
    train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
    model = TwoLayerSynaptic(num_inputs=7, num_hidden=h, num_outputs=1, batch_size=32)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, validation=False)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 1584, epochs: 18


100%|██████████| 18/18 [02:45<00:00,  9.22s/it]


Accuracy on validation dataset: 33.84704295896744%
MAE: 7.978202665456112, MSE:125.2646256544342,
RMSE: 11.192168049776335, Index of Agreement: 0.7878017387393628

------------- Random set: 1 -----------------
Hidden: 2095, epochs: 36


100%|██████████| 36/36 [05:40<00:00,  9.47s/it]


Accuracy on validation dataset: 38.952032363706415%
MAE: 6.822095941656749, MSE:97.07273593781754,
RMSE: 9.852549717601914, Index of Agreement: 0.847512653256542


### SNN with additional layer

In [27]:
for i in range(5):
    h = np.random.randint(300, 2500)
    e = np.random.randint(10, 40)
    s = np.random.randint(20, 30)
    b = np.random.choice([32, 64, 128])
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, steps: {s}, batch size: {b}')
    train_loader = data.DataLoader(train_dataset, batch_size=int(b), shuffle=True, drop_last=True)
    model = ThreeLayerLeaky(num_inputs=7, num_hidden=h, num_outputs=1, beta=0.95, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, validation=False)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 2308, epochs: 15, steps: 23, batch size: 64


100%|██████████| 15/15 [00:37<00:00,  2.52s/it]


Accuracy on validation dataset: 31.862839529955693%
MAE: 8.90247745257631, MSE:172.29357625224483,
RMSE: 13.126064766419708, Index of Agreement: 0.7158743392912515

------------- Random set: 1 -----------------
Hidden: 1800, epochs: 17, steps: 25, batch size: 32


100%|██████████| 17/17 [01:05<00:00,  3.87s/it]


Accuracy on validation dataset: 33.615873627432094%
MAE: 7.827615096054195, MSE:141.74367024973932,
RMSE: 11.90561507229842, Index of Agreement: 0.7272164758094452

------------- Random set: 2 -----------------
Hidden: 2001, epochs: 12, steps: 28, batch size: 32


...

Accuracy on validation dataset: 33.249855519167795%
MAE: 8.408341333416473, MSE:159.9792108328556,
RMSE: 12.648288849992936, Index of Agreement: 0.7320797242922241

------------- Random set: 3 -----------------
Hidden: 709, epochs: 37, steps: 26, batch size: 32


100%|██████████| 37/37 [02:21<00:00,  3.84s/it]


Accuracy on validation dataset: 33.365440184935466%
MAE: 8.081708734571945, MSE:142.30255020984026,
RMSE: 11.929063257852238, Index of Agreement: 0.7316616708075936

------------- Random set: 4 -----------------
Hidden: 2141, epochs: 23, steps: 21, batch size: 32


100%|██████████| 23/23 [01:28<00:00,  3.84s/it]


Accuracy on validation dataset: 34.07821229050279%
MAE: 7.9205683405701075, MSE:137.22574298786105,
RMSE: 11.714339204063585, Index of Agreement: 0.7561676444679714
